In [21]:
import re  # 正規表現ライブラリをインポート

def validate_and_correct_output(sub_stage):
    """
   sub_taskの形式を検証し、不正な形式の場合は修正する。
    """
    # 正規表現でTNM分類をチェック
    tnm_pattern = (
        r"(0|1)(\s(0|1)){7}"
    )
    if re.fullmatch(tnm_pattern, sub_stage):
        # 正しい形式の場合、そのまま返す
        return sub_stage
    else:
        # 正規表現パターンに一致する部分を検索
        match = re.search(tnm_pattern, sub_stage)
        if match:
            return match.group()  # 一致する部分文字列を返す
        else:
            return None  # 一致する部分がない場合はNoneを返す

tmp = '出力: 1 0 0 0 0 0 0 0'
print(validate_and_correct_output(tmp))

1 0 0 0 0 0 0 0


In [10]:
import pandas as pd

train_txt_path = '../radnlp_2024_train_val_20240731/ja/sub_task/train/sentences.csv'
df_train_txt = pd.read_csv(train_txt_path)

train_label_path = '../radnlp_2024_train_val_20240731/ja/sub_task/train/label.csv'
df_train_label = pd.read_csv(train_label_path)
df_train_label["sentence_index"] = df_train_label.groupby("id").cumcount()

#display(df_train_txt.head(10))
#display(df_train_label.head(10))

df_train = pd.merge(df_train_txt, df_train_label, on=["id", "sentence_index"])

few_shot_examples = []

def example(row):
    ex_dict = {}
    ex_dict["文章"] = row["text"]
    labels = []
    for col in df_train.columns[3:]:
       labels.append(row[col]) 
    ex_dict["出力"] =f"{labels[0]} {labels[1]} {labels[2]} {labels[3]} {labels[4]} {labels[5]} {labels[6]} {labels[7]}"
    few_shot_examples.append(ex_dict)

df_train.apply(example, axis=1)

#display(df_train_label.head())
print(few_shot_examples[:2])

#display(df_train)

[{'文章': '左上葉全体が無気肺になっています。', '出力': '0 0 0 1 0 0 0 0'}, {'文章': '左上葉気管支は閉塞して造影  CT  で増強効果の乏しい 74mm  の腫瘤があります。', '出力': '0 1 1 0 0 0 0 0'}]


In [22]:
%%time

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from dotenv import load_dotenv
from langchain.callbacks import tracing_enabled

import os
import pandas as pd
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

from langchain_anthropic import ChatAnthropic

from langsmith import Client, traceable

import datetime
import pytz
now = str(datetime.datetime.now(pytz.timezone('Asia/Tokyo')))[:-16]; now = now.replace(" ", "_").replace("-", "_").replace(":", "_")

load_dotenv()
ANTHROPIC_API_KEY  = os.getenv('ANTHROPIC_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# OpenAIのAPIキーを環境変数から取得
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = f"{now}_radnlp2024"
model_names = ['o1-preview-2024-09-12']  # ['gpt-4o-2024-05-13','gpt-4o-mini-2024-07-18','o1-preview-2024-09-12']
#model_names = ["claude-3-sonnet-20240229"]

# プロンプトテンプレートの読み込み
with open('../subtask_prompt.txt', 'r', encoding='utf-8') as file:
    subtask_prompt_text = file.read()

# FewShotPromptTemplateを作成
examples = [{"content": ex["文章"], "sub": ex["出力"]} for ex in few_shot_examples]

example_template = """
文章: {content}
出力: {sub}
"""

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["content", "sub"],
        template=example_template
    ),
    prefix=(
        "あなたは優秀な医師です。以下の文章全体を考慮し、指定された対象文について、以下のどのラベルに該当するかを判断してください。\n\n"
        "ラベル: \n{subtask_prompt_text}\n\n"
        "文章全体:\n{context}\n\n"
        "対象文:\n{sentence}\n\n"
        "次のトピックに該当する場合は「1」、該当しない場合は「0」としてください。\n"
        "1. omittable\n"
        "2. measure\n"
        "3. extension\n"
        "4. atelectasis\n"
        "5. satellite\n"
        "6. lymphadenopathy\n"
        "7. pleural\n"
        "8. distant\n\n"
    ),
    suffix=(
        "出力形式: 0または1をスペース区切りで出力してください。\n"
        "例: 1 0 0 0 0 0 0 0\n"
        "出力："
    ),
    input_variables=["context", "sentence", "subtask_prompt_text"]
)


# sentences.csv を読み込み
sentences_path = '../radnlp_2024_train_val_20240731/ja/sub_task/val/sentences.csv'
sentences = pd.read_csv(sentences_path)

for model_name in model_names:
    print(f"モデル {model_name} を使用して予測中...")
    output_csv = f'../sentence_predictions_{model_name}.csv'

    # LLMの設定
    if '4o' in model_name:
        llm = ChatOpenAI(temperature=0.7, model_name=model_name)
    else:
        llm = ChatOpenAI(temperature=1, model_name=model_name)

    # チェーンの作成
    chain = LLMChain(llm=llm, prompt=few_shot_prompt)

    results = []
    
    grouped = sentences.groupby('id')

    for group_id, group in grouped:
        # 文脈としてすべての文を結合
        context = "\n".join(group['text'])
        
        for _, row in group.iterrows():
            sentence = row['text']  # 対象の文
            try:
                # モデルに入力して予測を取得
                output = chain.run({"context": context, "sentence": sentence, "subtask_prompt_text": subtask_prompt_text}).strip()
                output = validate_and_correct_output(output)
                # 出力をスペースで分割して数値に変換
                predictions = list(map(int, output.split()))
                # 結果をリストに保存
                results.append({
                    "id": row['id'],
                    "sentence_index": row['sentence_index'],
                    "omittable": predictions[0],
                    "measure": predictions[1],
                    "extension": predictions[2],
                    "atelectasis": predictions[3],
                    "satellite": predictions[4],
                    "lymphadenopathy": predictions[5],
                    "pleural": predictions[6],
                    "distant": predictions[7],
                })
            except Exception as e:
                print(f"エラーが発生しました: ID={row['id']}, sentence_index={row['sentence_index']} - {e}")
                continue

    # DataFrameに変換してCSVファイルに保存
    results_df = pd.DataFrame(results)
    output_csv = f'../fewshot_sentence_classifications_{model_name}.csv'
    results_df.to_csv(output_csv, index=False)

    print(f"結果が{output_csv}に保存されました！")




モデル o1-preview-2024-09-12 を使用して予測中...
結果が../fewshot_sentence_classifications.csvに保存されました！
CPU times: user 11.7 s, sys: 448 ms, total: 12.2 s
Wall time: 2h 42min 42s


In [19]:
print("出力: 0 1 0 0 0 0 0 0".strip())
print(validate_and_correct_output("出力: 0 1 0 0 0 0 0 0"))

出力: 0 1 0 0 0 0 0 0
0 1 0 0 0 0 0 0
